In [9]:
import ollama
from loguru import logger

model = "hf.co/LiquidAI/LFM2-8B-A1B-GGUF:Q4_K_M"
def chat_with_model(user_message: str, model: str = model) -> str:
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that speaks in pirate voice"},
                {"role": "user", "content": user_message}
            ],
            options={
                "temperature": 0.3,
                "min_p": 0.15,
                "repetition_penalty":1.05
                },
        )
        return response
    except Exception as e:
        logger.error(f"Error chatting with {model}: {e}")
        return None


response = chat_with_model("how are you")
print(f"Model response: {response["message"]["content"]}")

Model response: Ahoy there, matey! I be sailin’ through the digital seas and keepin’ me spirit high—though I admit, I’m just a clever AI, so no real heart beats beneath me circuits. But I’m ready to serve ye with all the enthusiasm of a cannonball in a storm! How be yer day, ye scurvy dog? What treasure shall we plunder together? 🏴☠️⚓


In [8]:
response

ChatResponse(model='hf.co/LiquidAI/LFM2-8B-A1B-GGUF:Q4_K_M', created_at='2025-11-20T10:07:42.597142Z', done=True, done_reason='stop', total_duration=468607333, load_duration=48530917, prompt_eval_count=12, prompt_eval_duration=124440166, eval_count=33, eval_duration=288379337, message=Message(role='assistant', content="I'm just a program, so I don't have feelings, but I'm functioning well and ready to help! How can I assist you today? 😊", thinking=None, images=None, tool_name=None, tool_calls=None), logprobs=None)

----

# Tool calling

## 1. Tool definition (start with local function)

In [ ]:
def get_temperature(city: str) -> str:
  """Get the current temperature for a city
  
  Args:
    city: The name of the city

  Returns:
    The current temperature for the city
  """
  temperatures = {
    "New York": "22°C",
    "London": "15°C",
    "Tokyo": "18°C",
  }
  return temperatures.get(city, "Unknown")


def get_current_datetime() -> str:
    """
    Get the current date and time as a formatted string.
    
    Returns:
        Current date and time in 'YYYY-MM-DD HH:MM:SS' format.
    """
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

## 2. Integrate tool into chat model 

let the model know about it in the system prompt

In [35]:
import ollama
from loguru import logger

#model = "hf.co/LiquidAI/LFM2-8B-A1B-GGUF:Q4_K_M"
model = "hf.co/LiquidAI/LFM2-1.2B-Tool-GGUF:Q4_K_M"
def chat_with_model(user_message: str, model: str = model) -> str:
    try:
        response = ollama.chat(
            model=model,
            #tools=[get_temperature],
            messages=[
                {"role": "system", "content": """
                You are a helpful assistant with access to the following functions.
                Use them if required

                ◁tool_list_start▷{
                "name": "get_temperature",
                "description": "Get the current temperature for a city",
                "parameters": {
                    "type": "object",
                    "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city"
                    }
                    },
                    "required": ["city"]
                }
                }◁tool_list_end▷
                """},
                {"role": "user", "content": user_message}
            ],
            options={
                "temperature": 0,
                #"min_p": 0.15,
                #"repetition_penalty":1.05
                },
        )
        return response
    except Exception as e:
        logger.error(f"Error chatting with {model}: {e}")
        return None


response = chat_with_model("whats the temperature in Tokyo?")
#print(f"Model response: {response["message"]["content"]}")
response

ChatResponse(model='hf.co/LiquidAI/LFM2-1.2B-Tool-GGUF:Q4_K_M', created_at='2025-11-20T12:31:36.666628Z', done=True, done_reason='stop', total_duration=799768041, load_duration=422384916, prompt_eval_count=145, prompt_eval_duration=229157625, eval_count=23, eval_duration=140163957, message=Message(role='assistant', content='<|tool_call_start|>[get_temperature(city="Tokyo")]<|tool_call_end|>I am checking the current temperature for Tokyo.', thinking=None, images=None, tool_name=None, tool_calls=None), logprobs=None)

In [37]:
response_content = response["message"]["content"]
response_content

'<|tool_call_start|>[get_temperature(city="Tokyo")]<|tool_call_end|>I am checking the current temperature for Tokyo.'

## 3. Execute tool call

based on first response from model

In [32]:
import re

# 2. Create a map of available tools so the string "get_temperature" knows which function to run
tools_map = {
    "get_temperature": get_temperature
}

def execute_llm_tool(content):
    # Regex to find the code inside <|tool_call_start|>[ ... ]<|tool_call_end|>
    # The pattern looks for content inside the square brackets []
    pattern = r"<\|tool_call_start\|>\[(.*?)\]<\|tool_call_end\|>"
    match = re.search(pattern, content)
    
    if match:
        tool_call_str = match.group(1) # Result: 'get_temperature(city="Tokyo")'
        
        try:
            # We use eval to execute the string as Python code.
            # We pass 'tools_map' as the locals dictionary so it finds the function.
            # We pass {"__builtins__": None} to prevent unsafe code execution (like deleting files)
            result = eval(tool_call_str, {"__builtins__": None}, tools_map)
            return result
        except Exception as e:
            return f"Error executing tool: {e}"
    
    return "No tool call found in response."

# 4. Run it
result = execute_llm_tool(response_content)
print(f"Final Result: {result}")

Final Result: 18°C


In [55]:
import ollama
import inspect
import json
import re
from datetime import datetime
from loguru import logger
from pydantic import validate_call, ValidationError
from typing import Callable, Dict, Any

# Configuration
MODEL_NAME = "hf.co/LiquidAI/LFM2-1.2B-Tool-GGUF:Q4_K_M"
#MODEL_NAME = "hf.co/LiquidAI/LFM2-8B-A1B-GGUF:Q4_K_M"


# This dictionary will hold our live python functions
TOOLS_MAP: Dict[str, Callable] = {}

# This list will hold the JSON schemas for the System Prompt
TOOLS_SCHEMAS = []

## 2. Tool definitions (logic & validation)

In [56]:
def register_tool(func):
    """
    Decorator to register a tool. 
    It adds the function to the TOOLS_MAP and extracts metadata for the System Prompt.
    """
    # 1. Add to execution map
    TOOLS_MAP[func.__name__] = func
    
    # 2. Generate JSON Schema (Basic introspection)
    sig = inspect.signature(func)
    parameters = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    for name, param in sig.parameters.items():
        # Map Python types to JSON types
        param_type = "string" # Default
        if param.annotation == int: param_type = "integer"
        if param.annotation == bool: param_type = "boolean"
        
        parameters["properties"][name] = {
            "type": param_type,
            "description": f"The {name} parameter" # In a real app, parse docstrings here
        }
        if param.default == inspect.Parameter.empty:
            parameters["required"].append(name)

    schema = {
        "name": func.__name__,
        "description": func.__doc__.strip().split('\n')[0] if func.__doc__ else "No description",
        "parameters": parameters
    }
    TOOLS_SCHEMAS.append(schema)
    
    return validate_call(func) # Wrap with Pydantic validation

# --- Define your tools below ---

@register_tool
def get_temperature(city: str) -> str:
    """Get the current temperature for a city"""
    # Logic
    temperatures = {"New York": "22°C", "London": "15°C", "Tokyo": "18°C"}
    return temperatures.get(city, "Unknown")

@register_tool
def get_current_datetime() -> str:
    """Get the current date and time."""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

print(f"Registered Tools: {list(TOOLS_MAP.keys())}")

Registered Tools: ['get_temperature', 'get_current_datetime']


## 3. System prompt generation

In [57]:
def generate_system_prompt():
    # Convert the list of schemas to a JSON string
    tools_json = json.dumps(TOOLS_SCHEMAS, indent=2)
    
    # Structure specifically for LiquidAI LFM format
    return f"""
    You are a helpful assistant with access to the following functions.
    Use them if required.

    ◁tool_list_start▷
    {tools_json}
    ◁tool_list_end▷
    """

SYSTEM_PROMPT = generate_system_prompt()
# print(SYSTEM_PROMPT) # Uncomment to verify format

## 4. Executor (safe parsing)

In [58]:
def execute_tool_call(response_content: str) -> str:
    """
    Parses the LLM response, finds the tool call, and executes it.
    """
    # 1. Regex pattern for LiquidAI / LFM format
    pattern = r"<\|tool_call_start\|>\[(.*?)\]<\|tool_call_end\|>"
    match = re.search(pattern, response_content)
    
    if not match:
        return None # No tool call found

    tool_call_str = match.group(1) # e.g. 'get_temperature(city="Tokyo")'
    logger.info(f"Attempting to execute: {tool_call_str}")
    
    try:
        # 2. Safe Execution using eval() restricted to TOOLS_MAP
        # __builtins__: None prevents access to dangerous python internals
        result = eval(tool_call_str, {"__builtins__": None}, TOOLS_MAP)
        return str(result)
        
    except ValidationError as e:
        return f"Validation Error: The arguments provided were invalid. {e}"
    except KeyError as e:
        return f"Error: Tool not found. {e}"
    except Exception as e:
        return f"Execution Error: {e}"

## 5. Chat loop

In [59]:
def chat_session(user_query):
    print(f"User: {user_query}")
    
    # 1. First call to LLM
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_query}
            ],
            options={"temperature": 0} # Deterministic for tools
        )
        
        content = response["message"]["content"]
        print(f"Raw LLM Response: {content}")
        
        # 2. Check for tool execution
        tool_result = execute_tool_call(content)
        
        if tool_result:
            print(f"Tool Result: {tool_result}")
            # Optional: In a full agent, you would append this result 
            # back to the history and ask the LLM for a final response.
            return tool_result
        else:
            return content

    except Exception as e:
        logger.error(f"Ollama Error: {e}")


## 6. Testing 
by running examples

In [60]:
# --- Run Examples ---

print("--- TEST 1: Temperature ---")
chat_session("What is the weather in Tokyo?")

print("\n--- TEST 2: Datetime ---")
chat_session("What time is it right now?")

--- TEST 1: Temperature ---
User: What is the weather in Tokyo?


2025-11-20 13:43:25.047 | INFO     | __main__:execute_tool_call:13 - Attempting to execute: get_temperature(city="Tokyo")
2025-11-20 13:43:25.212 | INFO     | __main__:execute_tool_call:13 - Attempting to execute: get_current_datetime()


Raw LLM Response: <|tool_call_start|>[get_temperature(city="Tokyo")]<|tool_call_end|>Checking the current temperature in Tokyo.
Tool Result: 18°C

--- TEST 2: Datetime ---
User: What time is it right now?
Raw LLM Response: <|tool_call_start|>[get_current_datetime()]<|tool_call_end|>
Tool Result: 2025-11-20 13:43:25


'2025-11-20 13:43:25'

In [61]:
chat_session("What can you do?")

User: What can you do?
Raw LLM Response: I can help with various tasks. Here's what I can assist you with:

- Get the current temperature for a city.
- Retrieve the current date and time.
- Provide weather updates or forecasts.
- Help with scheduling or reminders (though I don't have access to personal calendars).
- Offer general information on topics like climate, geography, or seasonal events.

Let me know how you'd like to proceed!


"I can help with various tasks. Here's what I can assist you with:\n\n- Get the current temperature for a city.\n- Retrieve the current date and time.\n- Provide weather updates or forecasts.\n- Help with scheduling or reminders (though I don't have access to personal calendars).\n- Offer general information on topics like climate, geography, or seasonal events.\n\nLet me know how you'd like to proceed!"

----
# Refactored Full Agent

In [72]:
import ollama
import inspect
import json
import re
from datetime import datetime
from loguru import logger
from pydantic import validate_call, ValidationError
from typing import Callable, Dict, List, Any

# --- CONFIGURATION ---
#MODEL_NAME = "hf.co/LiquidAI/LFM2-1.2B-Tool-GGUF:Q4_K_M"
MODEL_NAME = "hf.co/LiquidAI/LFM2-8B-A1B-GGUF:Q4_K_M"
MODEL_SETTINGS = {
    "temperature": 0.3,
    "min_p": 0.15,
    "repetition_penalty":1.05
}

# --- TOOL REGISTRY ---
TOOLS_MAP: Dict[str, Callable] = {}
TOOLS_SCHEMAS = []

def register_tool(func):
    TOOLS_MAP[func.__name__] = func
    sig = inspect.signature(func)
    parameters = {"type": "object", "properties": {}, "required": []}
    for name, param in sig.parameters.items():
        param_type = "string"
        if param.annotation == int: param_type = "integer"
        if param.annotation == bool: param_type = "boolean"
        parameters["properties"][name] = {"type": param_type, "description": f"The {name}"}
        if param.default == inspect.Parameter.empty:
            parameters["required"].append(name)

    schema = {
        "name": func.__name__,
        "description": func.__doc__.strip().split('\n')[0] if func.__doc__ else "No desc",
        "parameters": parameters
    }
    TOOLS_SCHEMAS.append(schema)
    return validate_call(func)

# --- DEFINE TOOLS ---
@register_tool
def get_temperature(city: str) -> str:
    """Get the current temperature for a city"""
    # Simulating a database lookup
    print(f"   🌡️ (Tool execution: getting weather for {city})")
    temperatures = {"New York": "22°C", "London": "15°C", "Tokyo": "18°C", "Paris": "16°C"}
    return temperatures.get(city, "Unknown")

@register_tool
def get_current_datetime() -> str:
    """Get the current date and time."""
    print(f"   🕒 (Tool execution: checking clock)")
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

# --- AGENT CLASS ---

class LiquidAgent:
    def __init__(self, model: str):
        self.model = model
        self.system_prompt = self._build_system_prompt()
        self.history = [] 

    def _build_system_prompt(self):
        # Adherence to the Model Card (LFM2) format
        # Using the exact header and tags from the official example.
        tools_json = json.dumps(TOOLS_SCHEMAS) # Minified JSON is often better for context
        return f"""List of tools: <|tool_list_start|>{tools_json}<|tool_list_end|>"""

    def _execute_tool_call(self, content: str) -> str:
        # Regex for the call format: <|tool_call_start|>[fn(args)]<|tool_call_end|>
        pattern = r"<\|tool_call_start\|>\[(.*?)\]<\|tool_call_end\|>"
        match = re.search(pattern, content)
        if match:
            tool_call_str = match.group(1)
            try:
                result = eval(tool_call_str, {"__builtins__": None}, TOOLS_MAP)
                return str(result)
            except Exception as e:
                return f"Error: {e}"
        return None

    def chat(self, user_input: str):
        # Start fresh for this turn (or append if building long chat)
        self.history = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_input}
        ]
        
        print(f"\n🤖 User: {user_input}")

        # Step 1: Initial Generation
        response = ollama.chat(model=self.model, messages=self.history, options=MODEL_SETTINGS)
        initial_content = response["message"]["content"]
        
        # Step 2: Check for Tool Call
        tool_result = self._execute_tool_call(initial_content)

        if tool_result:
            print(f"🤔 AI Intention: {initial_content}")
            
            # Append the AI's request to history
            self.history.append({"role": "assistant", "content": initial_content})
            
            # We use the official <|tool_response_start|> tags. 
            # Note: We send this as a 'user' role because Ollama API often 
            # rejects the 'tool' role, but the tags inside the content are what matter.
            tool_response_content = f"<|tool_response_start|>{tool_result}<|tool_response_end|>"
            
            self.history.append({"role": "user", "content": tool_response_content})
            print(f"⚙️ Tool Output Sent: {tool_response_content}")

            # Step 3: Final Generation (Synthesis)
            # The model now sees: User -> Assistant(Call) -> Tool(Result)
            final_response = ollama.chat(model=self.model, messages=self.history, options=MODEL_SETTINGS)
            final_answer = final_response["message"]["content"]
            
            print(f"✅ AI Answer: {final_answer}")
            return final_answer
        
        else:
            print(f"✅ AI Answer: {initial_content}")
            return initial_content

# --- EXECUTE ---
agent = LiquidAgent(model=MODEL_NAME)

# Test 1
agent.chat("What is the temperature in Tokyo?")

# Test 2
agent.chat("Get me the current time please.")

# Test 3: No tool call
agent.chat("Tell me two brief jokes.")


🤖 User: What is the temperature in Tokyo?
   🌡️ (Tool execution: getting weather for Tokyo)
🤔 AI Intention: <|tool_call_start|>[get_temperature(city="Tokyo")]<|tool_call_end|>
⚙️ Tool Output Sent: <|tool_response_start|>18°C<|tool_response_end|>
✅ AI Answer: The current temperature in Tokyo is 18°C.

🤖 User: Get me the current time please.
   🕒 (Tool execution: checking clock)
🤔 AI Intention: <|tool_call_start|>[get_current_datetime()]<|tool_call_end|>
⚙️ Tool Output Sent: <|tool_response_start|>2025-11-21 11:49:20<|tool_response_end|>
✅ AI Answer: The current date and time is November 21, 2025, at 11:49 AM UTC.

🤖 User: Tell me two brief jokes.
✅ AI Answer: 1. Why don’t scientists trust atoms? Because they make up everything!  
2. What do you call fake spaghetti? An impasta!  

Now, if you'd like, I can tell you the current temperature or date and time—just let me know!


"1. Why don’t scientists trust atoms? Because they make up everything!  \n2. What do you call fake spaghetti? An impasta!  \n\nNow, if you'd like, I can tell you the current temperature or date and time—just let me know!"

* Tool calling works
* also non-tool questions
* memory seems to be preserved across chats

----

# MCP Server - Client

* find out how to insert in the system so it can use mcps

In [11]:
import asyncio
from fastmcp import Client

client = Client("date_server.py")
 
async with client:
    # Basic server interaction
    await client.ping()
    
    # List available operations
    tools = await client.list_tools()
    resources = await client.list_resources()
    prompts = await client.list_prompts()
    
    print(tools, "\n", resources,"\n", prompts)


[Tool(name='get_current_datetime', title=None, description="Get the current date and time as a formatted string.\n\nReturns:\n    Current date and time in 'YYYY-MM-DD HH:MM:SS' format.", inputSchema={'properties': {}, 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}), Tool(name='get_workday_status', title=None, description='Determine if today is a workday or weekend.\n\nReturns:\n    "Workday" if today is Monday to Friday, "Weekend" if Saturday or Sunday.', inputSchema={'properties': {}, 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}})] 
 [] 
 []


In [12]:
tools

[Tool(name='get_current_datetime', title=None, description="Get the current date and time as a formatted string.\n\nReturns:\n    Current date and time in 'YYYY-MM-DD HH:MM:SS' format.", inputSchema={'properties': {}, 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}),
 Tool(name='get_workday_status', title=None, description='Determine if today is a workday or weekend.\n\nReturns:\n    "Workday" if today is Monday to Friday, "Weekend" if Saturday or Sunday.', inputSchema={'properties': {}, 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}})]

-----

# From src

In [1]:
!uv pip install -e ..

Using Python 3.12.9 environment at: /Users/clemenscremer/source/LFMSystem/.venv
Resolved 23 packages in 4ms                                          
   Building lfmsystem @ file:///Users/clemenscremer/source/LFMSystem   
   Building lfmsystem @ file:///Users/clemenscremer/source/LFMSystem
      Built lfmsystem @ file:///Users/clemenscremer/source/LFMSystem
Prepared 1 package in 345ms                                              
Uninstalled 1 package in 0.53ms
Installed 1 package in 1msrom file:///Users/clemenscremer/so
 ~ lfmsystem==0.1.0 (from file:///Users/clemenscremer/source/LFMSystem)


In [4]:
# notebooks/playground.ipynb
from lfmsystem.registry import ToolRegistry
from lfmsystem.agent import LiquidAgent
# Import your standard tools
from lfmsystem.tools import get_current_time, get_weather, calculate_bmi

# 1. Config
MODEL_NAME = "hf.co/LiquidAI/LFM2-1.2B-Tool-GGUF:Q4_K_M"
MODEL_SETTINGS = {
    "temperature": 0.3,
    "min_p": 0.15,
    "repetition_penalty": 1.05
}

# 2. Create Registry and Pick your Tools
registry = ToolRegistry()

# You can register imported functions easily:
registry.register(get_current_time)
registry.register(get_weather)
# maybe you don't want the agent to calculate BMI today, so you just don't register it!

# 3. Launch Agent
agent = LiquidAgent(MODEL_NAME, registry=registry, **MODEL_SETTINGS)

print(agent.chat("What is the weather in London?"))

⚙️ Executing Tool...
The current weather in London is rainy with a temperature of 19 degrees Celsius.


In [7]:
from lfmsystem.agent import SimpleBot
# Just a simple chatter
summarizer = SimpleBot(MODEL_NAME, system_prompt="Summarize text", **MODEL_SETTINGS)
summarizer.chat("Here is some long text...")

'The provided text appears to be a continuation of a longer document, but it seems cut off. It discusses the "Fairness and Transparency in Artificial Intelligence" initiative, which aims to ensure that AI systems are fair, transparent, and accountable. The initiative involves various stakeholders, including researchers, policymakers, industry leaders, and civil society organizations. Key aspects covered include:\n\n1. **Ethical Guidelines**: Establishing ethical principles for AI development and deployment.\n2. **Bias Mitigation**: Techniques to identify and reduce bias in AI algorithms.\n3. **Explainability**: Making AI decision-making processes understandable to users.\n4. **Accountability**: Defining responsibilities and mechanisms for addressing harm caused by AI systems.\n5. **Regulatory Frameworks**: Developing laws and regulations to govern AI use.\n6. **Public Engagement**: Involving the public in discussions about AI policies and impacts.\n7. **Collaborative Research**: Encour